## Summing all the prime numbers below a given number

In [1]:
import time
import sys
import os

# Modificación ejercicio 3.2 d)
try:
    number = int(sys.argv[1])
except:
    print("womp womp")
    number = 2_500_000

if 'SLURM_CPUS_PER_TASK' in os.environ:
    cpus = int(os.environ['SLURM_CPUS_PER_TASK'])
    print("Detected %s CPUs through slurm"%cpus)
else:
    # None means that it will auto-detect based on os.cpu_count()
    # This is the same to put cpus = os.cpu_count()
    cpus = 4
    print("Running on default number of CPUs (Para este boletin: %s)"%cpus)

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


Running on default number of CPUs (Para este boletin: 4)
The prime sum below  2500000 is  219697708195  and the time taken is 4.788385073343913
The prime sum below  2500000 is  219697708195  and the time taken is 4.63 s ± 8.74 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


# Apartado 3.2
## Ejercicio A

In [2]:
import time
from numba import njit

# Simple code
@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

The prime sum below  2500000 is  219697708195  and the time taken is 0.6733713150024414
The prime sum below  2500000 is  219697708195  and the time taken is 186 ms ± 1.12 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


## Ejercicio B
Entiendo que sólo se hace en el bucle externo que usa `map`, si no habría que cambiar la función `sum_primes` a lo mismo.

In [3]:
import time
from numba import njit
import multiprocessing as mp

# Simple code
@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

suma = 0
N = 3 # number of loops

start = time.time()
with mp.Pool(cpus) as pool:
    suma = sum(pool.map(if_prime, range(number)))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

The prime sum below  2500000 is  219697708195  and the time taken is 0.10835552215576172
The prime sum below  2500000 is  219697708195  and the time taken is 187 ms ± 645 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


## Ejercicio C

In [4]:
import time
from numba import njit, prange
from numba import set_num_threads, get_num_threads

# Simple code
@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit(parallel=True)
def sum_primes(x):
    result = 0
    for i in prange(x):
        result += if_prime(i)
    return result

# Número de procesos (cpus)
set_num_threads(1 if cpus == None else cpus)

suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo, " with ",  get_num_threads)

The prime sum below  2500000 is  219697708195  and the time taken is 0.5606064796447754
The prime sum below  2500000 is  219697708195  and the time taken is 70.2 ms ± 80.2 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)  with  <function get_num_threads at 0x7f23047379c0>


## Ejercicio D y E
La salida completa del *notebook* tras lanzar el *batch* es:
```text
#############################################################################
PARA 1000000 NÚMERO DE ELEMENTOS
#############################################################################
#############################################################################
CON 1 HILOS
#############################################################################
Detected 1 CPUs through slurm
The prime sum below  1000000 is  37550402023  and the time taken is 2.236467123031616
The prime sum below  1000000 is  37550402023  and the time taken is 2.21 s ± 10.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  1000000 is  37550402023  and the time taken is 4.224985122680664
The prime sum below  1000000 is  37550402023  and the time taken is 101 ms ± 54.7 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
The prime sum below  1000000 is  37550402023  and the time taken is 0.2191141446431478
The prime sum below  1000000 is  37550402023  and the time taken is 101 ms ± 30 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  1000000 is  37550402023  and the time taken is 0.35459184646606445
The prime sum below  1000000 is  37550402023  and the time taken is 115 ms ± 11.9 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)  with  <function get_num_threads at 0x7f30baf567a0>
#############################################################################
CON 2 HILOS
#############################################################################
Detected 2 CPUs through slurm
The prime sum below  1000000 is  37550402023  and the time taken is 2.2614523569742837
The prime sum below  1000000 is  37550402023  and the time taken is 2.22 s ± 14.9 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  1000000 is  37550402023  and the time taken is 0.7278684775034586
The prime sum below  1000000 is  37550402023  and the time taken is 101 ms ± 69.2 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
The prime sum below  1000000 is  37550402023  and the time taken is 0.12059736251831055
The prime sum below  1000000 is  37550402023  and the time taken is 101 ms ± 8.25 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  1000000 is  37550402023  and the time taken is 0.3510126272837321
The prime sum below  1000000 is  37550402023  and the time taken is 70.5 ms ± 10.2 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)  with  <function get_num_threads at 0x7f2f8770a7a0>
#############################################################################
CON 4 HILOS
#############################################################################
Detected 4 CPUs through slurm
The prime sum below  1000000 is  37550402023  and the time taken is 2.3051615556081138
The prime sum below  1000000 is  37550402023  and the time taken is 2.21 s ± 9.06 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  1000000 is  37550402023  and the time taken is 0.5595460732777914
The prime sum below  1000000 is  37550402023  and the time taken is 101 ms ± 362 ns per loop (mean ± std. dev. of 2 runs, 10 loops each)
The prime sum below  1000000 is  37550402023  and the time taken is 0.09296703338623047
The prime sum below  1000000 is  37550402023  and the time taken is 101 ms ± 11.9 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  1000000 is  37550402023  and the time taken is 0.35279122988382977
The prime sum below  1000000 is  37550402023  and the time taken is 37.6 ms ± 10 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)  with  <function get_num_threads at 0x7f5f3620a7a0>
#############################################################################
CON 8 HILOS
#############################################################################
Detected 8 CPUs through slurm
The prime sum below  1000000 is  37550402023  and the time taken is 2.244454781214396
The prime sum below  1000000 is  37550402023  and the time taken is 2.21 s ± 9.3 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  1000000 is  37550402023  and the time taken is 0.5623962084452311
The prime sum below  1000000 is  37550402023  and the time taken is 101 ms ± 3.04 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
The prime sum below  1000000 is  37550402023  and the time taken is 0.0874005953470866
The prime sum below  1000000 is  37550402023  and the time taken is 101 ms ± 10 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  1000000 is  37550402023  and the time taken is 0.35200150807698566
The prime sum below  1000000 is  37550402023  and the time taken is 26.9 ms ± 3.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)  with  <function get_num_threads at 0x7fdef5ef27a0>
#############################################################################
PARA 10000000 NÚMERO DE ELEMENTOS
#############################################################################
#############################################################################
CON 1 HILOS
#############################################################################
Detected 1 CPUs through slurm
The prime sum below  10000000 is  3203324994356  and the time taken is 59.00966993967692
The prime sum below  10000000 is  3203324994356  and the time taken is 57.8 s ± 352 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  10000000 is  3203324994356  and the time taken is 5.062475840250651
The prime sum below  10000000 is  3203324994356  and the time taken is 2.54 s ± 184 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  10000000 is  3203324994356  and the time taken is 1.798353672027588
The prime sum below  10000000 is  3203324994356  and the time taken is 2.54 s ± 115 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  10000000 is  3203324994356  and the time taken is 4.6157753467559814
The prime sum below  10000000 is  3203324994356  and the time taken is 2.87 s ± 28.6 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)  with  <function get_num_threads at 0x7f39110aa7a0>
#############################################################################
CON 2 HILOS
#############################################################################
Detected 2 CPUs through slurm
The prime sum below  10000000 is  3203324994356  and the time taken is 59.420599699020386
The prime sum below  10000000 is  3203324994356  and the time taken is 57.9 s ± 332 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  10000000 is  3203324994356  and the time taken is 5.063827037811279
The prime sum below  10000000 is  3203324994356  and the time taken is 2.54 s ± 330 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  10000000 is  3203324994356  and the time taken is 1.0087740421295166
The prime sum below  10000000 is  3203324994356  and the time taken is 2.54 s ± 33.7 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  10000000 is  3203324994356  and the time taken is 4.600144386291504
The prime sum below  10000000 is  3203324994356  and the time taken is 1.79 s ± 760 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)  with  <function get_num_threads at 0x7fc27f22e7a0>
#############################################################################
CON 4 HILOS
#############################################################################
Detected 4 CPUs through slurm
The prime sum below  10000000 is  3203324994356  and the time taken is 58.9006179968516
The prime sum below  10000000 is  3203324994356  and the time taken is 57.8 s ± 353 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  10000000 is  3203324994356  and the time taken is 5.376497109731038
The prime sum below  10000000 is  3203324994356  and the time taken is 2.54 s ± 210 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  10000000 is  3203324994356  and the time taken is 0.5873077710469564
The prime sum below  10000000 is  3203324994356  and the time taken is 2.54 s ± 193 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  10000000 is  3203324994356  and the time taken is 4.602925856908162
The prime sum below  10000000 is  3203324994356  and the time taken is 957 ms ± 168 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)  with  <function get_num_threads at 0x7f896a0b67a0>
#############################################################################
CON 8 HILOS
#############################################################################
Detected 8 CPUs through slurm
The prime sum below  10000000 is  3203324994356  and the time taken is 59.465562423070274
The prime sum below  10000000 is  3203324994356  and the time taken is 58.2 s ± 371 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  10000000 is  3203324994356  and the time taken is 5.100383996963501
The prime sum below  10000000 is  3203324994356  and the time taken is 2.54 s ± 309 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  10000000 is  3203324994356  and the time taken is 0.39584962526957196
The prime sum below  10000000 is  3203324994356  and the time taken is 2.54 s ± 25.2 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  10000000 is  3203324994356  and the time taken is 4.615272045135498
The prime sum below  10000000 is  3203324994356  and the time taken is 493 ms ± 198 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)  with  <function get_num_threads at 0x7fef5f6da7a0>
```

Para mejor legibilidad y análisis de los resultados:

| Valor  | Optimización                         | N Hilos | Map tiempo avg. $\approx$ (ms) | sum_primes tiempo avg. $\approx$ (ms)  |
| :---   | :---                                 | :---:   | ---:                           | ---:                                   |
| $10^6$ | Sin optimización                     | –       | $2250$                         | $2250$                                 |
|        | @njit                                | –       | $550$                          | $101$                                  |
|        | @njit + multiprocessing              | 1       | $219$                          | –                                      |
|        |                                      | 2       | $120$                          | –                                      |
|        |                                      | 4       | $92.96$                        | –                                      |
|        |                                      | 8       | $87.4$                         | –                                      |
|        | @njit + paralelismo en sum_primes    | 1       | –                              | $115$                                  |
|        |                                      | 2       | –                              | $70.5$                                 |
|        |                                      | 4       | –                              | $37.6$                                 |
|        |                                      | 8       | –                              | $26.9$                                 |
| $10^7$ | Sin optimización                     | –       | $59500$                        | $57800$                                |
|        | @njit                                | –       | $5000$                         | $2540$                                 |
|        | @njit + multiprocessing              | 1       | $1798$                         | –                                      |
|        |                                      | 2       | $1008$                         | –                                      |
|        |                                      | 4       | $587$                          | –                                      |
|        |                                      | 8       | $395$                          | –                                      |
|        | @njit + paralelismo en sum_primes    | 1       | –                              | $2870$                                 |
|        |                                      | 2       | –                              | $1790$                                 |
|        |                                      | 4       | –                              | $957$                                  |
|        |                                      | 8       | –                              | $493$                                  |

A continuación se muestran los speedups finales para `map`:

| Valor  | Método                    | N Hilos | Tiempo (ms) | Speedup      |
| :---   | :---                      | :---:   | ---:        | ---:         |
| $10^6$ | @njit                     | –       | $550$       | $4.1\times$  |
|        | @njit + multiprocessing   | 8       | $87.4$      | $25.7\times$ |
| $10^7$ | @njit                     | –       | $5000$      | $11.9\times$ |
|        | @njit + multiprocessing   | 8       | $395$       | $151\times$  |

Y para `sum_primes`:

| Valor  | Método         | N Hilos | Tiempo (ms) | Speedup      |
| :---   | :---           | :---:   | ---:        | ---:         |
| $10^6$ | @njit          | –       | $101$       | $22.3\times$ |
|        | @njit + prange | 8       | $26.9$      | $83.6\times$ |
| $10^7$ | @njit          | –       | $2540$      | $22.8\times$ |
|        | @njit + prange | 8       | $493$       | $117\times$  |



En primer lugar, para la versión sin optimización, tanto al usar `map` como la función `sum_primes`, los tiempos de ejecución son elevados y del mismo orden de magnitud.

Al introducir *numba* en modo **secuencial** con `@njit`, se observa una mejora significativa en ambos enfoques. No obstante, el beneficio es mucho más notable en la función `sum_primes`, que pasa a ejecutarse en tiempos muy inferiores a los obtenidos mediante `map`. Esto se debe a que `sum_primes` presenta un bucle simple, ideal para ser optimizado por Numba.

Cuando se emplea *multiprocessing* con `Pool`, la mejora se observa únicamente en el caso de `map`, ya que esta versión permite repartir el trabajo de forma natural entre procesos. La función `sum_primes` no se ha modificado, ya que sería igual que el bloque de `map`. A medida que aumenta el número de núcleos, el tiempo de ejecución disminuye, aunque el escalado no es lineal debido al coste de creación de procesos y a la comunicación entre ellos.

Finalmente, la versión paralelizada con *numba* aplicada a `sum_primes`, que ha ofrecido mejores resultados con $10^6$ elementos y algo inferiores a *multiprocessing* con $10^7$. Podemos concluir que, para **volumenes** más **grandes** de datos, sería conveniente usar **numba y multiprocessing** pudiendo llegar a obtener hasta una mejora en un factor de 150 frente al 117 de usar sólo *numba* con *@njit* y *prange*